In [4]:
# visualize_zones_in_service_area_map.py
# 交互式地图：Waymo 边界 + 相交的 TAZ（面）+ 运营范围内的 zones（点）
# 仅使用 TAZ，不再使用 census block

from pathlib import Path
import json
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
import folium
from folium.plugins import MarkerCluster

# ========= 可配置 =========
SERVICE_AREA_GEOJSON = r"../data/raw data/waymo_atx_2025-03-03.geojson"
TAZ_SHP              = r"../data/raw data/tl_2011_48_taz10.shp"
ZONES_CSV_IN         = r"../data/zones.csv"              # 列: zone,lat,lon（zone 与 TAZCE10 对齐）
OUT_HTML             = r"../data/viz/zones_in_service_area_map.html"

# 统一 TAZ 编码宽度（常见 6；如你的流水线是 8 位，如 "00000450"，改为 8）
ZONE_ZFILL = 6

# 仅为加速的县过滤（Travis/Williamson/Hays/Bastrop/Caldwell/Burnet）
COUNTY_FILTER = {"453", "491", "209", "021", "055", "053"}  # 置为 set() 不过滤

# 相交比例阈值（以 TAZ 自身面积为分母）。0 表示只要有交就保留
OVERLAP_THRESHOLD = 0.5

# 若服务区是折线（LINESTRING），在米制投影下的缓冲半径（把线转成“面”）
LINE_BUFFER_METERS = 30.0

# 等积投影用于面积计算
EQUAL_AREA_EPSG = 5070
# =========================


def load_service_area(path: str) -> gpd.GeoDataFrame:
    """把服务区数据（面或线）合并成一个清洁的面多边形，CRS=EPSG:4326。"""
    sa_raw = gpd.read_file(path)
    sa_raw = (sa_raw.to_crs(4326) if sa_raw.crs is not None
              else sa_raw.set_crs(4326, allow_override=True))

    geom = sa_raw.geometry
    is_line_like = geom.geom_type.isin(["LineString", "MultiLineString"]).all()

    if is_line_like:
        sa_m = sa_raw.to_crs(3857)
        geom_m = unary_union(sa_m.geometry).buffer(LINE_BUFFER_METERS)
        sa_poly = gpd.GeoDataFrame(geometry=[geom_m], crs=3857).to_crs(4326)
    else:
        geom_u = unary_union(geom).buffer(0)  # 修复自交
        sa_poly = gpd.GeoDataFrame(geometry=[geom_u], crs=4326)

    if not isinstance(sa_poly.geometry.iloc[0], (Polygon, MultiPolygon)):
        raise ValueError("服务区几何未能转换为多边形，请检查 GeoJSON。")
    return sa_poly[["geometry"]]


def _detect_taz_id_col(gdf: gpd.GeoDataFrame) -> str:
    """优先识别 TAZCE10；否则尝试一组常见 TAZ 字段。"""
    cols = {c.lower(): c for c in gdf.columns}
    for c in ["tazce10", "taz", "taz_id", "tazid", "taz_code", "tazcode", "taz_no", "taznum", "taz_number"]:
        if c in cols:
            return cols[c]
    # 实在没有，才退而求其次用 geoid10/geoid（不推荐）
    for c in ["geoid10", "geoid"]:
        if c in cols:
            return cols[c]
    # 兜底：纯数字且长度>=4
    for c in gdf.columns:
        s = gdf[c].astype(str)
        if s.str.fullmatch(r"\d{4,}").all():
            return c
    raise ValueError(f"未找到 TAZ 字段；实际列：{list(gdf.columns)}")


def _detect_county_col(gdf: gpd.GeoDataFrame) -> str | None:
    """尝试检测县字段（用于可选 COUNTY_FILTER）。"""
    for c in gdf.columns:
        cl = c.lower()
        if re.fullmatch(r"countyfp(\d{2})?", cl) or "county" in cl:
            return c
    return None


def load_taz_in_area(shp_path: str, sa_wgs84: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    读取 TAZ 多边形，与服务区做空间相交（支持 overlap 阈值）。
    返回 columns: [zone, geometry, overlap_ratio]，CRS=4326；zone=TAZCE10 左补零到 ZONE_ZFILL。
    """
    gdf = gpd.read_file(shp_path)
    gdf = (gdf.to_crs(4326) if gdf.crs is not None else gdf.set_crs(4326, allow_override=True))

    taz_col     = _detect_taz_id_col(gdf)
    county_col  = _detect_county_col(gdf)

    if COUNTY_FILTER and county_col:
        gdf = gdf[gdf[county_col].astype(str).isin(COUNTY_FILTER)].copy()

    # bbox 粗筛
    minx, miny, maxx, maxy = sa_wgs84.total_bounds
    gdf = gdf.cx[minx:maxx, miny:maxy].copy()
    gdf["geometry"] = gdf.buffer(0)

    # 等积投影计算相交占比（修复弃用：使用 union_all()）
    sa_ea  = sa_wgs84.to_crs(EQUAL_AREA_EPSG).geometry.union_all()
    gdf_ea = gdf.to_crs(EQUAL_AREA_EPSG)

    hits = gdf_ea[gdf_ea.intersects(sa_ea)].copy()
    if hits.empty:
        return gpd.GeoDataFrame(columns=["zone","geometry","overlap_ratio"], crs=4326)

    inter = hits.geometry.intersection(sa_ea)
    hits["intersect_area"] = inter.area
    hits["poly_area"]      = hits.geometry.area
    hits["overlap_ratio"]  = (hits["intersect_area"] / hits["poly_area"]).clip(0, 1)

    keep = hits[hits["overlap_ratio"] >= OVERLAP_THRESHOLD].copy().to_crs(4326)
    keep = keep.rename(columns={taz_col: "zone"})[["zone", "geometry", "overlap_ratio"]]
    keep["zone"] = keep["zone"].astype(str).str.strip().str.zfill(ZONE_ZFILL)
    return keep


def load_zones_points(zones_csv: str, keep_ids: set[str], taz_keep_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    读入 zones.csv 的点（zone,lat,lon）。
    - 先按 keep_ids 过滤（zone 与 TAZCE10 对齐）
    - 若缺经纬度，用 TAZ 面的代表点补齐
    """
    df = pd.read_csv(zones_csv, dtype={"zone": str})
    if "zone" not in df.columns:
        raise ValueError("zones.csv 需包含列 'zone'。")

    df["zone"] = df["zone"].astype(str).str.strip().str.zfill(ZONE_ZFILL)
    df = df[df["zone"].isin(keep_ids)].copy()

    # 有坐标则用；无坐标用 TAZ 代表点补齐
    has_latlon = {"lat","lon"}.issubset(df.columns)
    pts_list = []

    if has_latlon:
        df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
        df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
        pts_nonnull = df.dropna(subset=["lat","lon"]).copy()
        if not pts_nonnull.empty:
            g_nonnull = gpd.GeoDataFrame(
                pts_nonnull[["zone","lat","lon"]],
                geometry=gpd.points_from_xy(pts_nonnull["lon"], pts_nonnull["lat"]),
                crs=4326
            )
            pts_list.append(g_nonnull[["zone","geometry"]])

    # 对缺失 zone，从 TAZ 面代表点补齐
    missing = keep_ids - set(df.dropna(subset=["lat","lon"])["zone"]) if has_latlon else keep_ids
    if missing:
        cent = taz_keep_gdf[taz_keep_gdf["zone"].isin(missing)].copy()
        cent["geometry"] = cent.geometry.representative_point()
        pts_list.append(cent[["zone","geometry"]])

    # 汇总
    if pts_list:
        gdf_pts = pd.concat(pts_list, ignore_index=True)
        gdf_pts = gpd.GeoDataFrame(gdf_pts, geometry="geometry", crs=4326)
        # 同一 zone 可能多点：保留全部用于可视化聚合
        return gdf_pts[["zone","geometry"]]

    # 极端情况：CSV 完全没有坐标且为空集
    return gpd.GeoDataFrame(columns=["zone","geometry"], crs=4326)


def to_geojson_dict(gdf: gpd.GeoDataFrame) -> dict:
    return json.loads(gdf.to_json())


def add_service_area_layer(m: folium.Map, sa: gpd.GeoDataFrame):
    gj = to_geojson_dict(sa)
    folium.GeoJson(
        gj,
        name="Service Area",
        style_function=lambda x: {"color": "#d62728", "weight": 3, "fillOpacity": 0.05},
        highlight_function=lambda x: {"weight": 4, "color": "#ff9896"},
    ).add_to(m)


def add_taz_layer(m: folium.Map, taz_keep: gpd.GeoDataFrame):
    if taz_keep.empty:
        return
    gj = to_geojson_dict(taz_keep[["zone","overlap_ratio","geometry"]])
    folium.GeoJson(
        gj,
        name="TAZ in Area",
        style_function=lambda feat: {
            "color": "#1f77b4",
            "weight": 1,
            "fillColor": "#1f77b4",
            "fillOpacity": min(0.65, 0.15 + 0.5*float(feat["properties"].get("overlap_ratio", 0))),
        },
        highlight_function=lambda x: {"weight": 2, "color": "#aec7e8"},
        tooltip=folium.GeoJsonTooltip(
            fields=["zone","overlap_ratio"],
            aliases=["TAZ","overlap"],
            localize=True
        ),
    ).add_to(m)


def add_zones_points_layer(m: folium.Map, zones_pts: gpd.GeoDataFrame):
    grp = folium.FeatureGroup(name="Zones (points)", show=True)
    cluster = MarkerCluster(name="Zones cluster", disableClusteringAtZoom=15)
    for _, r in zones_pts.iterrows():
        p = r.geometry
        folium.CircleMarker(
            location=[p.y, p.x],
            radius=3,
            color="#2ca02c",
            fill=True,
            fill_opacity=0.9,
            weight=1,
            popup=folium.Popup(html=f"TAZ: {r['zone']}", max_width=260),
            tooltip=f"TAZ: {r['zone']}",
        ).add_to(cluster)
    cluster.add_to(grp)
    grp.add_to(m)


def main():
    # 1) 服务区面
    sa = load_service_area(SERVICE_AREA_GEOJSON)

    # 2) 与 TAZ 相交
    taz_keep = load_taz_in_area(TAZ_SHP, sa)
    if taz_keep.empty:
        raise SystemExit("没有与服务范围相交的 TAZ，多半是边界/县过滤过严或路径不对。")

    keep_ids = set(taz_keep["zone"].astype(str))

    # 3) zones 点位（来自 CSV；缺失经纬度时用 TAZ 代表点）
    zones_pts = load_zones_points(ZONES_CSV_IN, keep_ids, taz_keep)

    # 4) 地图
    minx, miny, maxx, maxy = sa.total_bounds
    center = [(miny+maxy)/2, (minx+maxx)/2]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodbpositron")

    add_service_area_layer(m, sa)
    add_taz_layer(m, taz_keep)
    add_zones_points_layer(m, zones_pts)

    folium.LayerControl(collapsed=False).add_to(m)
    m.fit_bounds([[miny, minx], [maxy, maxx]])

    Path(OUT_HTML).parent.mkdir(parents=True, exist_ok=True)
    m.save(OUT_HTML)
    print(f"Saved map -> {OUT_HTML}")
    print(f"zone points plotted: {len(zones_pts)} | TAZ polygons drawn: {len(taz_keep)}")

if __name__ == "__main__":
    main()


Saved map -> ../data/viz/zones_in_service_area_map.html
zone points plotted: 221 | TAZ polygons drawn: 221
